- reference : 
    - [Youtube : Julia Tutorial 2022 Korea 줄리아 언어 소개](https://www.youtube.com/watch?v=wjLXmz4D9VQ&t=2131s)

In [ ]:
# Distributed computing in Julia
- 김기백 박사 (Argonne National Laboratory)
---

- 줄리아 `Distributed` 패키지에서 스케줄링도 가능
- `MPI` 패키지 + shell 스크립트를 이용하면 `C`만큼의 퍼포먼스가 나옴.
- `C`에 비해 Julia의 편의성이 월등하고, 퍼포먼스는 주의 깊은 코딩으로 달성 가능하다.
- multiple dispatch를 통해 자료형을 쉽게 변환하고 최적화된 연산을 수행할 수 있다.

In [ ]:
# Julia + Deep Learning
- Flux.jl 
    - 줄리아 100% 연산 라이브러리
    - 가볍고 기본에 충실한 딥러닝 프레임워크(Jax와 유사)
    - 복잡한 모델이나 다른 기능을 원한다면 Flux.jl 기반 다른 패키지를 사용하거나 직접 작성
        - Transformer : Transformers.jl
        - Geometric Deep learning : Flux.jl
    - Flux.jl을 쓰면 좋은 경우
        - 시뮬레이션과 결합한 딥러닝
        - 내부 Deep Learning 코드를 건드려야하는 경우
          ex - LSTM 수식 자체를 수정해야하는 경우
        - Automatic differentiation에 관련된 프로젝트
        - 파이썬보다 parallel computing이 더 간단
    - Flux.jl을 쓰면 안 좋은 경우
        - 일반적인 Deep Learning만 돌리는 경우
        - CPU, GPU외에 장비를 사용하는 경우
        - 디버깅이 어려움(문서도 아직 갈 길이 멀고, 자잘한 버그도 많음.)

In [ ]:
## Basic Neural Network
using Pkg

Pkg.add("Flux")

In [2]:
## Basic NN
using Flux

NN1 = Chain(Dense(10, 5, tanh),
    Dense(5,5, tanh),
    Dense(5,2))

NN1(rand(10))

LoadError: ArgumentError: Package Flux not found in current path:
- Run `import Pkg; Pkg.add("Flux")` to install the Flux package.


In [ ]:
## Custom Activation Function

NN2 = Chain(Dense(10, 5, x -> x^2),
    Dense(5,5, relu),
    Dense(5, 2))

NN2(rand(10))

In [ ]:
#### pytorch에서 이를 구현하려면 좀 더 복잡

import torch
from torch import nn

model1 = nn.Sequential(
    nn.Linear(10, 5),
    nn.Tanh(),
    nn.Linear(5,5),
    nn.Tanh(),
    nn.Linear(5,2)v
)


In [ ]:
## Train Neural Network
- 모델 training을 위한 기본 요소
    - Objective function
    - parameters of the model.
    - data
    - optimaizaer
- train!(loss, params, data, opt; cb)

In [ ]:
- Loss (MSE)
    - loss(x, y) = Flux.Losses.mse(NN1(x), y)
- Parameters
    - ps = Flux.params(NN1)
- Optimisers
    - opt = Flux.Descent()
- Data
    - Julia는 Column-major
    - Shuffling, mini-batch iteration 기능을 지니는 DataLoader 사용 가능

- Training
    for epoch_idx in 1:epoch_size
        Flux.train!(loss, ps, trian_data, opt)
        println(sum(loss.(test_x, test_y)))
    end

In [ ]:
## Train wiht GPU
- Array를 GPU로 변환
- Flux model도 GPU로 변환 가능
    - RNN의 경우 순차적으로 돌아야 되기 때문에 for 구문을 써야함.
    - 각 배치에 대해 broadcasting을 활용해서 쉽게 적용 가능.